In [1059]:
import wrangle as w
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
import xgboost as xgb
from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from sklearn.ensemble import BaggingClassifier
from imblearn.ensemble import EasyEnsembleClassifier

In [1254]:
df = w.get_cws_data()
df

,id,total_charges,amount_paid,open,charge_code,description,prop_id,charge_name,sStatus,rent,term,monthly_inc,GuarantorRequired,total_inc,Recommendation,age,risk_score,reason,bad_resident
0,3946,75.00,75.00,0,131,"Uncllctbl key, remote fees",136,"Uncllctbl key, remote fees",Current,1311,15,52500,False,666000,Accept,33,745,Insufficient period of Residence History,1
1,3962,13.66,13.66,0,112,Water 8/1/2021-9/1/2021,136,Utility-water charges,Current,1661,12,65000,False,889992,Accept with Conditions (Extra Deposit),27,758,No Credit Experience,0
2,4050,46.17,46.17,0,155,Damage charges - Clean,136,Damage charges,Current,1412,12,48000,False,576000,Accept with Conditions (Extra Deposit),33,666,Insufficient period of Residence History,1
3,4948,9.76,9.76,0,111,Sewer Charge,140,Utility-sewer charges,Current,1377,18,72000,False,0,A-Criminal History Meets Requirements,39,0,Meets All Property Requirements,0
4,5001,6.85,6.85,0,113,Sewer Base Charge,140,Utility-water flat base charges,Current,2199,13,0,True,54996,Accept with Conditions (Guarantor),23,752,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,74033,1439.37,1439.37,0,74,Rent for 29 days,85,Rent,Current,1489,12,0,True,0,Accept with Conditions (Guarantor),24,708,Insufficient Income to Support Rent,0
4995,74056,150.00,150.00,0,162,Deposit Waiver Fee (Amount subject to screenin...,152,Deposit waiver fee,Current,1859,13,10000,False,121776,Accept,32,786,Meets All Property Requirements,0
4996,74230,5.00,5.00,0,186,Convenience Fee,61,OSP resident convenience fee,Current,1439,12,4350,False,57432,Accept with Conditions (Extra Deposit),36,691,Limited period of Residence History,0
4997,74231,5.00,5.00,0,186,Convenience Fee (ray farmer),83,OSP resident convenience fee,Current,1739,12,7916,False,94992,Accept,47,770,Meets All Property Requirements,0


In [3]:
df.bad_resident.value_counts()

0    4808
1     191
Name: bad_resident, dtype: int64

In [4]:
train, y_train, y_val, y_test, train_scaled, val_scaled, test_scaled = w.model_prep(df)

AttributeError: 'DataFrame' object has no attribute 'fit_resample'

In [6]:
dt = DecisionTreeClassifier(max_depth=12, random_state=91)

In [7]:
dt = dt.fit(train_scaled, y_train)

In [8]:
train_pred = dt.predict(train_scaled)

In [9]:
print(classification_report(y_train, train_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2692
           1       1.00      0.80      0.89       107

    accuracy                           0.99      2799
   macro avg       1.00      0.90      0.94      2799
weighted avg       0.99      0.99      0.99      2799



In [10]:
labels = sorted(y_train.unique())

In [11]:
pd.DataFrame(confusion_matrix(y_train, train_pred), index=labels, columns=labels)

,0,1
0,2692,0
1,21,86


In [12]:
val_pred = dt.predict(val_scaled)

In [13]:
print(classification_report(y_val, val_pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      1154
           1       0.12      0.13      0.12        46

    accuracy                           0.93      1200
   macro avg       0.54      0.55      0.54      1200
weighted avg       0.93      0.93      0.93      1200



In [14]:
pd.DataFrame(confusion_matrix(y_val, val_pred), index=labels, columns=labels)

,0,1
0,1109,45
1,40,6


In [15]:
df_majority = train[train.bad_resident==0]
df_minority = train[train.bad_resident==1]
len(df_minority)

107

In [16]:
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=214,    # to match majority class
                                 random_state=91) # reproducible results

In [17]:
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

In [18]:
df_upsampled.bad_resident.value_counts()

0    2692
1     214
Name: bad_resident, dtype: int64

In [19]:
y_up = df_upsampled.bad_resident

In [20]:
df_upsampled = df_upsampled.drop(columns = 'bad_resident')

In [21]:
df_upsampled[['rent', 'monthly_inc','total_inc', 'age', 'risk_score']] = StandardScaler().fit_transform(df_upsampled[['rent', 'monthly_inc','total_inc', 'age', 'risk_score']])

In [22]:
df_upsampled

,rent,term,monthly_inc,total_inc,age,risk_score,prop_id_Arizona,prop_id_California,prop_id_Colorado,prop_id_Georgia,...,Recommendation_A-Criminal History Meets Requirements,Recommendation_A-Meets Requirements,Recommendation_Accept,Recommendation_Accept with Conditions (Extra Deposit),Recommendation_Accept with Conditions (Guarantor),Recommendation_Accept with Conditions (Max Deposit),Recommendation_Accept with Extra Security Deposit,Recommendation_Guarantor Not Qualified,Recommendation_Qualified Guarantor,Recommendation_REJECT
2707,-0.107014,12,-0.273994,-0.365509,-0.510785,-2.297009,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4983,0.101885,12,0.037528,-0.215222,-0.151858,0.601310,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4959,-0.049567,9,0.219365,0.542769,0.386531,0.375669,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4492,-0.263688,14,-0.147078,-0.078422,-0.331321,0.745253,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
995,0.258559,13,0.141369,-0.140734,-0.151858,0.519612,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,-0.524811,15,0.051373,-0.189435,-0.510785,0.029427,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
154,0.441345,12,-0.166346,-0.222465,-0.600516,0.255068,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
405,0.336896,12,-0.435525,-0.452922,-1.138906,0.667446,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
11,-0.060012,18,-0.016244,-0.009145,-0.421053,-2.297009,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


In [23]:
rf.fit(df_upsampled, y_up)

NameError: name 'rf' is not defined

In [ ]:
val_pred = rf.predict(val_scaled)

In [ ]:
print(classification_report(y_val, val_pred))

In [ ]:
pd.DataFrame(confusion_matrix(y_val, val_pred), index=labels, columns=labels)

In [ ]:
# train, y_train, y_val, y_test, train_scaled, val_scaled, test_scaled = w.model_prep(df_upsampled)

In [ ]:
dt = DecisionTreeClassifier(max_depth=8, random_state=91)

In [ ]:
dt = dt.fit(train_scaled, y_train)

In [ ]:
train_pred = dt.predict(train_scaled)

In [ ]:
print(classification_report(y_train, train_pred))

In [ ]:
pd.DataFrame(confusion_matrix(y_train, train_pred), index=labels, columns=labels)

In [ ]:
val_pred = dt.predict(val_scaled)

In [ ]:
print(classification_report(y_val, val_pred))

In [ ]:
pd.DataFrame(confusion_matrix(y_val, val_pred), index=labels, columns=labels)

In [45]:
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight= 'balanced', 
                            criterion='entropy',
                            min_samples_leaf=3,
                            n_estimators=100,
                            max_depth=5, 
                            random_state=91)

In [25]:
rf.fit(df_upsampled, y_up)

RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=5, min_samples_leaf=3, random_state=91)

In [26]:
train_pred = rf.predict(train_scaled)

In [27]:
print(classification_report(y_train, train_pred))

              precision    recall  f1-score   support

           0       0.98      0.82      0.90      2692
           1       0.13      0.66      0.22       107

    accuracy                           0.82      2799
   macro avg       0.56      0.74      0.56      2799
weighted avg       0.95      0.82      0.87      2799



In [28]:
val_pred = rf.predict(val_scaled)

In [29]:
print(classification_report(y_val, val_pred))

              precision    recall  f1-score   support

           0       0.98      0.82      0.89      1154
           1       0.11      0.54      0.18        46

    accuracy                           0.81      1200
   macro avg       0.54      0.68      0.53      1200
weighted avg       0.94      0.81      0.86      1200



In [30]:
pd.DataFrame(confusion_matrix(y_val, val_pred), index=labels, columns=labels)

,0,1
0,941,213
1,21,25


In [994]:
params = {'max_depth': [3, 4, 5, 6, 7, 8],
          'min_samples_leaf': [1, 3, 5],
         'n_estimators':[100, 250, 500],
         'class_weight': ['balanced'],
         'bootstrap': [True],
         'criterion':['gini', 'entropy'],
         'random_state': [91]}
rfG= RandomForestClassifier()
grid = GridSearchCV(rfG, params, cv=3, scoring = 'recall')
grid.fit(train_up, y_up)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True], 'class_weight': ['balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 4, 5, 6, 7, 8],
                         'min_samples_leaf': [1, 3, 5],
                         'n_estimators': [100, 250, 500],
                         'random_state': [91]},
             scoring='recall')

In [995]:
results = grid.cv_results_
test_scores = results['mean_test_score']
test_scores

array([0.65384935, 0.65014565, 0.64644195, 0.64644195, 0.65014565,
       0.65014565, 0.64644195, 0.65389097, 0.65389097, 0.63520599,
       0.63899293, 0.63899293, 0.63146067, 0.63524761, 0.63528922,
       0.6389097 , 0.63895131, 0.63899293, 0.63520599, 0.61656263,
       0.61290054, 0.62409488, 0.62039118, 0.61660424, 0.63520599,
       0.62034956, 0.62039118, 0.60162297, 0.59047025, 0.59413233,
       0.60915522, 0.59791927, 0.59791927, 0.62030795, 0.60536829,
       0.60911361, 0.58668331, 0.58668331, 0.59042863, 0.58676654,
       0.59787765, 0.59787765, 0.6127757 , 0.6127757 , 0.60907199,
       0.57548897, 0.57182688, 0.56437786, 0.58302122, 0.57927591,
       0.5755722 , 0.60158136, 0.61281731, 0.58297961, 0.6426134 ,
       0.64640033, 0.64269663, 0.6463171 , 0.64640033, 0.64269663,
       0.65006242, 0.64640033, 0.64269663, 0.63516438, 0.63516438,
       0.63895131, 0.63886808, 0.63146067, 0.63895131, 0.64627549,
       0.63520599, 0.63895131, 0.62030795, 0.61656263, 0.62401

In [996]:
params = results['params']

In [997]:
for p, s in zip(params, test_scores):
    p['score'] = s

pd.DataFrame(params).sort_values(by='score', ascending = False).head(30)

,bootstrap,class_weight,criterion,max_depth,min_samples_leaf,n_estimators,random_state,score
7,True,balanced,gini,3,5,250,91,0.653891
8,True,balanced,gini,3,5,500,91,0.653891
0,True,balanced,gini,3,1,100,91,0.653849
4,True,balanced,gini,3,3,250,91,0.650146
5,True,balanced,gini,3,3,500,91,0.650146
1,True,balanced,gini,3,1,250,91,0.650146
60,True,balanced,entropy,3,5,100,91,0.650062
2,True,balanced,gini,3,1,500,91,0.646442
3,True,balanced,gini,3,3,100,91,0.646442
6,True,balanced,gini,3,5,100,91,0.646442


In [989]:
cv = xgb.XGBClassifier(scale_pos_weight = 20, n_estimators = 200, max_depth = 4, max_leaves = 3, learning_rate = .2, 
                       min_child_weight = 1, 
                       eval_metric = roc_auc_score, early_stopping_rounds = 1)

In [990]:
cv = cv.fit(X = train_up, y = y_up, eval_set=[(train_up, y_up), (val_scaled, y_val)])

[0]	validation_0-logloss:0.67577	validation_0-roc_auc_score:0.80753	validation_1-logloss:0.68570	validation_1-roc_auc_score:0.64345


In [991]:
val_preds = cv.predict(val_scaled)

In [992]:
print(classification_report(y_val, val_preds))

              precision    recall  f1-score   support

           0       0.98      0.61      0.75      1154
           1       0.06      0.65      0.11        46

    accuracy                           0.61      1200
   macro avg       0.52      0.63      0.43      1200
weighted avg       0.94      0.61      0.72      1200



In [993]:
pd.DataFrame(confusion_matrix(y_val, val_preds), index=labels, columns=labels)

,0,1
0,699,455
1,16,30


In [998]:
sm = SMOTENC(categorical_features= list(range(6,24)), 
             random_state = 91, sampling_strategy = .1)

In [999]:
train_up, y_up = sm.fit_resample(train_scaled, y_train)

In [1046]:
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight= 'balanced', 
                            criterion='entropy',
                            min_samples_leaf=3,
                            n_estimators=200,
                            max_depth=4, 
                            random_state=91)

In [1047]:
rf.fit(train_up, y_up)

RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=4, min_samples_leaf=3, n_estimators=200,
                       random_state=91)

In [1048]:
val_pred = rf.predict(val_scaled)

In [1049]:
print(classification_report(y_val, val_pred))

              precision    recall  f1-score   support

           0       0.98      0.81      0.88      1154
           1       0.11      0.61      0.19        46

    accuracy                           0.80      1200
   macro avg       0.55      0.71      0.54      1200
weighted avg       0.95      0.80      0.86      1200



In [1050]:
pd.DataFrame(confusion_matrix(y_val, val_pred), index=labels, columns=labels)

,0,1
0,930,224
1,18,28


In [1051]:
sm1 = ADASYN(sampling_strategy = .1)

In [1011]:
train_sm, y_sm = sm1.fit_resample(train_scaled, y_train)

In [1012]:
rf.fit(train_sm, y_sm)

RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=4, min_samples_leaf=3, n_estimators=200,
                       random_state=91)

In [1013]:
val_pred = rf.predict(val_scaled)

In [1014]:
print(classification_report(y_val, val_pred))

              precision    recall  f1-score   support

           0       0.98      0.73      0.84      1154
           1       0.09      0.65      0.15        46

    accuracy                           0.73      1200
   macro avg       0.53      0.69      0.50      1200
weighted avg       0.95      0.73      0.81      1200



In [1015]:
pd.DataFrame(confusion_matrix(y_val, val_pred), index=labels, columns=labels)

,0,1
0,842,312
1,16,30


In [511]:
y_up.value_counts()

0    2692
1     269
Name: bad_resident, dtype: int64

In [1016]:
342/(858+342)

0.285

In [1053]:
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight= 'balanced', 
                            criterion='entropy',
                            min_samples_leaf=3,
                            n_estimators=200,
                            max_depth=4, 
                            random_state=91)

In [ ]:
params = {'max_depth': [3, 4, 5, 6, 7, 8],
          'min_samples_leaf': [1, 3, 5],
         'n_estimators':[100, 250, 500],
         'class_weight': ['balanced'],
         'bootstrap': [True],
         'criterion':['gini', 'entropy'],
         'random_state': [91]}
bcG= BaggingClassifier()
grid = GridSearchCV(rfG, params, cv=3, scoring = 'recall')
grid.fit(train_up, y_up)

In [1054]:
bc = BaggingClassifier()

In [1055]:
bc = bc.fit(train_up, y_up)

In [1057]:
val_bc = bc.predict(val_scaled)

In [1058]:
print(classification_report(y_val, val_bc))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1154
           1       0.00      0.00      0.00        46

    accuracy                           0.95      1200
   macro avg       0.48      0.50      0.49      1200
weighted avg       0.92      0.95      0.94      1200



In [1203]:
ez = EasyEnsembleClassifier(n_estimators = 10, sampling_strategy = .5, random_state = 91)

In [1204]:
ez = ez.fit(train_scaled, y_train)

In [1205]:
ez_pred = ez.predict(val_scaled)

In [1206]:
print(classification_report(y_val, ez_pred))

              precision    recall  f1-score   support

           0       0.98      0.83      0.90      1154
           1       0.13      0.63      0.21        46

    accuracy                           0.82      1200
   macro avg       0.55      0.73      0.56      1200
weighted avg       0.95      0.82      0.87      1200



In [1207]:
pd.DataFrame(confusion_matrix(y_val, ez_pred), index=labels, columns=labels)

,0,1
0,955,199
1,17,29


In [1208]:
ez_test = ez.predict(test_scaled)

In [1209]:
print(classification_report(y_test, ez_test))

              precision    recall  f1-score   support

           0       0.98      0.81      0.89       962
           1       0.10      0.55      0.17        38

    accuracy                           0.80      1000
   macro avg       0.54      0.68      0.53      1000
weighted avg       0.95      0.80      0.86      1000



In [1210]:
pd.DataFrame(confusion_matrix(y_test, ez_test), index=labels, columns=labels)

,0,1
0,780,182
1,17,21


In [1224]:
def randomforestNC(train, y_train, x, y):
    sm = SMOTENC(categorical_features= list(range(6,24)), 
             random_state = 91, sampling_strategy = .1)
    train_up, y_up = sm.fit_resample(train, y_train)
    
    rf = RandomForestClassifier(bootstrap=True, 
                            class_weight= 'balanced', 
                            criterion='entropy',
                            min_samples_leaf=3,
                            n_estimators=200,
                            max_depth=4, 
                            random_state=91)
    rf = rf.fit(train_up, y_up)
    train_pred = rf.predict(train_up)
    y_pred = rf.predict(x)
    class_report_train = classification_report(y_up, train_pred)
    class_report_val = classification_report(y, y_pred)
    # Extract the recall for each class
    recall_train = pd.Series(class_report_train.split()[6:13:5], name='train_log')
    recall_validate = pd.Series(class_report_val.split()[6:13:5], name='validate_log')
    df = pd.concat([recall_train, recall_validate], axis=1)
    df = pd.DataFrame(df, columns= ['train_log', 'validate_log'])
    return df
    

In [1225]:
def randomforestSMOTE(train, y_train, x, y):
    sm = SMOTE(random_state = 91, sampling_strategy = .1)
    train_up, y_up = sm.fit_resample(train, y_train)
    
    rf = RandomForestClassifier(bootstrap=True, 
                            class_weight= 'balanced', 
                            criterion='entropy',
                            min_samples_leaf=3,
                            n_estimators=200,
                            max_depth=4, 
                            random_state=91)
    rf = rf.fit(train_up, y_up)
    train_pred = rf.predict(train_up)
    y_pred = rf.predict(x)
    class_report_train = classification_report(y_up, train_pred)
    class_report_val = classification_report(y, y_pred)
    # Extract the recall for each class
    recall_train = pd.Series(class_report_train.split()[6:13:5], name='train_log')
    recall_validate = pd.Series(class_report_val.split()[6:13:5], name='validate_log')
    df = pd.concat([recall_train, recall_validate], axis=1)
    df = pd.DataFrame(df, columns= ['train_log', 'validate_log'])
    return df
    

In [1228]:
def randomforestADASYN(train, y_train, x, y):
    ad = ADASYN(random_state = 91, sampling_strategy = .1)
    train_up, y_up = ad.fit_resample(train, y_train)
    
    rf = RandomForestClassifier(bootstrap=True, 
                            class_weight= 'balanced', 
                            criterion='entropy',
                            min_samples_leaf=3,
                            n_estimators=200,
                            max_depth=4, 
                            random_state=91)
    rf = rf.fit(train_up, y_up)
    train_pred = rf.predict(train_up)
    y_pred = rf.predict(x)
    class_report_train = classification_report(y_up, train_pred)
    class_report_val = classification_report(y, y_pred)
    # Extract the recall for each class
    recall_train = pd.Series(class_report_train.split()[6:13:5], name='train_log')
    recall_validate = pd.Series(class_report_val.split()[6:13:5], name='validate_log')
    df = pd.concat([recall_train, recall_validate], axis=1)
    df = pd.DataFrame(df, columns= ['train_log', 'validate_log'])
    return df
    

In [1240]:
def xgboostSMOTE(train, y_train, x, y):
    sm = SMOTE(random_state = 91, sampling_strategy = .1)
    train_up, y_up = sm.fit_resample(train, y_train)
    cv = xgb.XGBClassifier(scale_pos_weight = 20, n_estimators = 200, max_depth = 4, max_leaves = 3, learning_rate = .2, 
                       min_child_weight = 1, 
                       eval_metric = roc_auc_score, early_stopping_rounds = 1)
    cv = cv.fit(X = train_up, y = y_up, eval_set=[(train_up, y_up), (val_scaled, y_val)])
    y_pred = cv.predict(x)
    train_pred = cv.predict(train_up)
    class_report_train = classification_report(y_up, train_pred)
    class_report_val = classification_report(y, y_pred)
    # Extract the recall for each class
    recall_train = pd.Series(class_report_train.split()[6:13:5], name='train_log')
    recall_validate = pd.Series(class_report_val.split()[6:13:5], name='validate_log')
    df = pd.concat([recall_train, recall_validate], axis=1)
    df = pd.DataFrame(df, columns= ['train_log', 'validate_log'])
    return df
    

In [1241]:
def xgboostSMOTENC(train, y_train, x, y):
    sm = SMOTENC(categorical_features= list(range(6,24)), 
             random_state = 91, sampling_strategy = .1)    
    train_up, y_up = sm.fit_resample(train, y_train)
    cv = xgb.XGBClassifier(scale_pos_weight = 20, n_estimators = 200, max_depth = 4, max_leaves = 3, learning_rate = .2, 
                       min_child_weight = 1, 
                       eval_metric = roc_auc_score, early_stopping_rounds = 1)
    cv = cv.fit(X = train_up, y = y_up, eval_set=[(train_up, y_up), (val_scaled, y_val)])
    y_pred = cv.predict(x)
    train_pred = cv.predict(train_up)
    class_report_train = classification_report(y_up, train_pred)
    class_report_val = classification_report(y, y_pred)
    # Extract the recall for each class
    recall_train = pd.Series(class_report_train.split()[6:13:5], name='train_log')
    recall_validate = pd.Series(class_report_val.split()[6:13:5], name='validate_log')
    df = pd.concat([recall_train, recall_validate], axis=1)
    df = pd.DataFrame(df, columns= ['train_log', 'validate_log'])
    return df

In [1246]:
def xgboostADASYN(train, y_train, x, y):
    ad = ADASYN(random_state = 91, sampling_strategy = .1)
    train_up, y_up = ad.fit_resample(train, y_train)
    cv = xgb.XGBClassifier(scale_pos_weight = 20, n_estimators = 200, max_depth = 4, max_leaves = 3, learning_rate = .2, 
                            min_child_weight = 1, 
                            eval_metric = roc_auc_score, early_stopping_rounds = 1)
    cv = cv.fit(X = train_up, y = y_up, eval_set=[(train_up, y_up), (val_scaled, y_val)])
    y_pred = cv.predict(x)
    train_pred = cv.predict(train_up)
    class_report_train = classification_report(y_up, train_pred)
    class_report_val = classification_report(y, y_pred)
    # Extract the recall for each class
    recall_train = pd.Series(class_report_train.split()[6:13:5], name='train_log')
    recall_validate = pd.Series(class_report_val.split()[6:13:5], name='validate_log')
    df = pd.concat([recall_train, recall_validate], axis=1)
    df = pd.DataFrame(df, columns= ['train_log', 'validate_log'])
    return df

In [1251]:
def ensemble(train, y_train, x, y):
    ez = EasyEnsembleClassifier(n_estimators = 10, sampling_strategy = .5, random_state = 91)
    ez = ez.fit(train, y_train)
    y_pred = ez.predict(x)
    train_pred = ez.predict(train)
    class_report_train = classification_report(y_train, train_pred)
    class_report_val = classification_report(y, y_pred)
    # Extract the recall for each class
    recall_train = pd.Series(class_report_train.split()[6:13:5], name='train_log')
    recall_validate = pd.Series(class_report_val.split()[6:13:5], name='validate_log')
    df = pd.concat([recall_train, recall_validate], axis=1)
    df = pd.DataFrame(df, columns= ['train_log', 'validate_log'])
    return df

In [1255]:
train_scaled

,rent,term,monthly_inc,total_inc,age,risk_score,prop_id_Arizona,prop_id_California,prop_id_Colorado,prop_id_Georgia,...,Recommendation_A-Criminal History Meets Requirements,Recommendation_A-Meets Requirements,Recommendation_Accept,Recommendation_Accept with Conditions (Extra Deposit),Recommendation_Accept with Conditions (Guarantor),Recommendation_Accept with Conditions (Max Deposit),Recommendation_Accept with Extra Security Deposit,Recommendation_Guarantor Not Qualified,Recommendation_Qualified Guarantor,Recommendation_REJECT
2707,-0.090190,12,-0.334993,-0.378361,-0.513961,-2.308439,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4983,0.122051,12,0.057028,-0.221645,-0.155214,0.593710,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4959,-0.031824,9,0.285852,0.568771,0.382907,0.367771,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4492,-0.249372,14,-0.175281,-0.078992,-0.334587,0.737843,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
995,0.281233,13,0.187701,-0.143971,-0.155214,0.511904,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,0.307763,5,-0.088165,-0.267677,0.024160,-0.111376,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2512,1.262851,12,0.018406,-0.599730,0.203534,0.574232,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
443,0.466944,14,-0.146243,-0.157578,-0.872708,0.021071,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4028,-0.488144,13,-0.010640,-0.006752,-0.962395,0.562546,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
